In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
pwd

'c:\\Users\\DELL\\Downloads\\github\\Kidney-disease-classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path
#making an entity

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from med_classifier.constants import *

In [6]:
from med_classifier.utils.common import read_yaml,create_directories

In [ ]:
#update the config manager in src config- making the main tower to read the yaml files
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    #doubt
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [ ]:
#update the components-downloading the data and extracting zip file
import os
import zipfile
import gdown
from med_classifier import logger
from med_classifier.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''
      #getting data from url
        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
#doubt
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [13]:
#step 7 pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-27 04:38:20,665: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-27 04:38:20,679: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-27 04:38:20,684: INFO: common: created directory at: artifacts]
[2025-07-27 04:38:20,686: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-27 04:38:20,690: INFO: 2299023065: Downloading data from https://drive.google.com/file/d/1835klpvgNBp_bYvJzEv4QOBAaAHcz9yX/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1835klpvgNBp_bYvJzEv4QOBAaAHcz9yX
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1835klpvgNBp_bYvJzEv4QOBAaAHcz9yX&confirm=t&uuid=f19713a4-f107-4e90-8aa3-afe917121b63
To: c:\Users\DELL\Downloads\github\Kidney-disease-classification\artifacts\data_ingestion\data.zip
100%|██████████| 941M/941M [01:58<00:00, 7.92MB/s] 

[2025-07-27 04:40:22,912: INFO: 2299023065: Downloaded data from https://drive.google.com/file/d/1835klpvgNBp_bYvJzEv4QOBAaAHcz9yX/view?usp=drive_link into file artifacts/data_ingestion/data.zip]
